<div class="alert alert-block alert-success">
    <p>  <b>Automotive Brand Affinity and Attribute Perception Analysis: </b></p>
    
<b>In this project, we utilise concepts of Association Theory and NLP to answer:</b>
    
   1.  Which entry level luxury sedan car brands are mostly talked about among customers? 
   2. Which  car brands do consumers most talk about together?   
   3. What are the attributes a particular brand is mostly associated with by the customers?
   
The entire project has been split into six different tasks for the ease of execution.

 1. Web Scrapping using Selenium
 2. Data Cleaning
 3. Identify the Top 5 brands
 4. Identify the brands being talked about together
 5. Identify the top 5 most frequently mentioned attributes of cars in the discussions
 6. Identify how strongly are these attributes associated with the car brands
</div>

### Task 1:  Web Scrapping using Selenium
>Sracpe 5000 comments from the Edmunds discussion forum:
https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans

In [10]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
#WebDriver is a browser automation framework that works with open source APIs.
#The framework operates by accepting commands, sending those commands to a browser, and interacting with applications.
chrome_options = webdriver.ChromeOptions()
#headless means running chrome with chrome.exe
chrome_options.add_argument('--headless')
import pandas as pd

chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [2]:
#creating new instance of chrome
wd = webdriver.Chrome(options=chrome_options)

#website to scrape
wd.get("https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans")


In [3]:
df_comments = pd.DataFrame(columns=['date', 'message'])

for i in range (1,101):
    wd.get("https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p"+str(i))
    ids = wd.find_elements(by=By.XPATH, value = '//*[contains(@id,"Comment_")]')
    comment_ids = []

    #for every path obtained in ids, extract the 'id' attribute because that has the comment ids.
    for i in ids:
         comment_ids.append(i.get_attribute('id'))
    
    for i in comment_ids:
        #user_id=wd.find_element(by=By.XPATH, value ='//*[@id="' + i +'"]/div/div[2]/div[1]/span[1]/a[2]')
        #userid=user_id.text
    
        comment_date=wd.find_element(by=By.XPATH, value ='//*[@id="'+ i +'"]/div/div[2]/div[2]/span/a/time')
        date = comment_date.get_attribute('title')
    
        comment=wd.find_element(by=By.XPATH,value='//*[@id="'+ i +'"]/div/div[3]/div/div[1]')
        user_comment=comment.text
    
        df_comments.loc[len(df_comments)] = [date,user_comment]
    if(len(df_comments)>=5000):
        break

In [4]:
len(df_comments)
df_comments.head()

,date,message
0,"March 24, 2002 9:54PM",I personally think that with a few tweaks the ...
1,"March 24, 2002 11:06PM",I am debating a new purchase and these two are...
2,"March 25, 2002 9:02AM","Great handling, RWD, excellent engine and the ..."
3,"March 25, 2002 3:04PM",And no manual tranny. That may not matter to y...
4,"March 25, 2002 4:44PM",One beauty of BMW 3 Series is that there are s...


In [5]:
#exporting the file to csv
df_comments.to_csv('Car_Reviews.csv')

### Task 2: Data Cleaning:
   >1. Remove stop words
   >2. Tokenise
   >3. Convert all to Lower Case

In [235]:
df=pd.read_csv('Car_Reviews.csv')
df.head()

,Unnamed: 0,date,message
0,0,"March 24, 2002 9:54PM",I personally think that with a few tweaks the ...
1,1,"March 24, 2002 11:06PM",I am debating a new purchase and these two are...
2,2,"March 25, 2002 9:02AM","Great handling, RWD, excellent engine and the ..."
3,3,"March 25, 2002 3:04PM",And no manual tranny. That may not matter to y...
4,4,"March 25, 2002 4:44PM",One beauty of BMW 3 Series is that there are s...


In [236]:
df = df.drop(columns=['Unnamed: 0'])
df.head()

,date,message
0,"March 24, 2002 9:54PM",I personally think that with a few tweaks the ...
1,"March 24, 2002 11:06PM",I am debating a new purchase and these two are...
2,"March 25, 2002 9:02AM","Great handling, RWD, excellent engine and the ..."
3,"March 25, 2002 3:04PM",And no manual tranny. That may not matter to y...
4,"March 25, 2002 4:44PM",One beauty of BMW 3 Series is that there are s...


In [237]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anmolagrawal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [238]:
#removing stop words from the comment
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    filtered_words=[]
    for i in words:
        if i.lower() not in stop_words:
            filtered_words.append(i.lower())
    return filtered_words
df['without sw'] = df_comments['message'].map(remove_stopwords)

In [239]:
df.head()

,date,message,without sw
0,"March 24, 2002 9:54PM",I personally think that with a few tweaks the ...,"[personally, think, tweaks, c320, could, also,..."
1,"March 24, 2002 11:06PM",I am debating a new purchase and these two are...,"[debating, new, purchase, two, top, list., pri..."
2,"March 25, 2002 9:02AM","Great handling, RWD, excellent engine and the ...","[great, handling,, rwd,, excellent, engine, be..."
3,"March 25, 2002 3:04PM",And no manual tranny. That may not matter to y...,"[manual, tranny., may, matter, you,, certainly..."
4,"March 25, 2002 4:44PM",One beauty of BMW 3 Series is that there are s...,"[one, beauty, bmw, 3, series, many, models, ch..."


### Task 3: Identify the Top 5 brands
> Find the top five brands that are being talked about in these discussions. In our data, we might have mentions of a car model instead of a brand, therefore, we first replace model names with the corrosponding brand in our messages. We have the car_models_and_brands.csv which has the mapping of the models to their brands.

In [240]:
brand_model_mapping=pd.read_csv('car_models_and_brands.csv')
brand_model_mapping.head(10)

,Brand,Model
0,acura,integra
1,acura,legend
2,acura,vigor
3,acura,rlx
4,acura,ilx
5,acura,mdx
6,acura,rdx
7,acura,tsx
8,acura,rsx
9,audi,quattro


In [241]:
model_to_brand = dict(zip(brand_model_mapping['Model'], brand_model_mapping['Brand']))
model_to_brand

{'integra': 'acura',
 'legend': 'honda',
 'vigor': 'acura',
 'rlx': 'acura',
 'ilx': 'acura',
 'mdx': 'acura',
 'rdx': 'acura',
 'tsx': 'acura',
 'rsx': 'acura',
 'quattro': 'audi',
 'a4': 'audi',
 'a6': 'audi',
 'a5': 'audi',
 'a7': 'audi',
 'q3': 'audi',
 'q5': 'audi',
 'q7': 'audi',
 'a8': 'audi',
 'a3': 'audi',
 '3-series': 'bmw',
 '5-series': 'bmw',
 '6-series': 'bmw',
 '4-series': 'bmw',
 'xdrive': 'bmw',
 'alpina': 'bmw',
 '330ci': 'bmw',
 '328i': 'bmw',
 '330c': 'bmw',
 '335i': 'bmw',
 '335d': 'bmw',
 '528i': 'bmw',
 '535i': 'bmw',
 '128i': 'bmw',
 '640i': 'bmw',
 '3er': 'bmw',
 '5er': 'bmw',
 'm5': 'bmw',
 'm3': 'bmw',
 'm6': 'bmw',
 'bmws': 'bmw',
 'bmwesque': 'bmw',
 'park avenue': 'buick',
 'la crosse': 'buick',
 'lacrosse': 'buick',
 'century': 'toyota',
 'enclave': 'buick',
 'encore': 'buick',
 'regal': 'buick',
 'buicks': 'buick',
 'escalade': 'cadillac',
 'srx': 'cadillac',
 'xts': 'cadillac',
 'grand blazer': 'chevrolet',
 'trailblazer': 'chevrolet',
 'silverado': 'che

In [242]:
#Replace the model name with brand name in the message

def model_rep(text):
    new_list=[]
    for i in text:
        if i in model_to_brand:
            new_list.append(model_to_brand[i])
        else:
            new_list.append(i)
    return new_list
    
    
df['brand_replaced_list'] = df['without sw'].map(model_rep)

In [243]:
df.head()

,date,message,without sw,brand_replaced_list
0,"March 24, 2002 9:54PM",I personally think that with a few tweaks the ...,"[personally, think, tweaks, c320, could, also,...","[personally, think, tweaks, c320, could, also,..."
1,"March 24, 2002 11:06PM",I am debating a new purchase and these two are...,"[debating, new, purchase, two, top, list., pri...","[debating, new, purchase, two, top, list., pri..."
2,"March 25, 2002 9:02AM","Great handling, RWD, excellent engine and the ...","[great, handling,, rwd,, excellent, engine, be...","[great, handling,, rwd,, excellent, engine, be..."
3,"March 25, 2002 3:04PM",And no manual tranny. That may not matter to y...,"[manual, tranny., may, matter, you,, certainly...","[manual, tranny., may, matter, you,, certainly..."
4,"March 25, 2002 4:44PM",One beauty of BMW 3 Series is that there are s...,"[one, beauty, bmw, 3, series, many, models, ch...","[one, beauty, bmw, 3, series, many, models, ch..."


In [244]:
#Next, we identify the brand mentions in each message. Count each brand mention only once.

def brand(text):
    new_list=[]
    for i in text:
        if i in list(model_to_brand.values()) and i not in new_list: 
            new_list.append(i)
        else:
            pass
    return new_list       

df['brand_list'] = df['brand_replaced_list'].map(brand)

In [245]:
# assigning count of 1 for every brand in each post.
def f(text):
    brand_count={}
    
    for i in text:
             brand_count[i]=1       
            
    return   brand_count     
        
df['brand_count']=df['brand_list'].map(f)

In [246]:
df.head()

,date,message,without sw,brand_replaced_list,brand_list,brand_count
0,"March 24, 2002 9:54PM",I personally think that with a few tweaks the ...,"[personally, think, tweaks, c320, could, also,...","[personally, think, tweaks, c320, could, also,...","[mercedes, audi, bmw, acura, lexus, buick, toy...","{'mercedes': 1, 'audi': 1, 'bmw': 1, 'acura': ..."
1,"March 24, 2002 11:06PM",I am debating a new purchase and these two are...,"[debating, new, purchase, two, top, list., pri...","[debating, new, purchase, two, top, list., pri...","[honda, lexus]","{'honda': 1, 'lexus': 1}"
2,"March 25, 2002 9:02AM","Great handling, RWD, excellent engine and the ...","[great, handling,, rwd,, excellent, engine, be...","[great, handling,, rwd,, excellent, engine, be...",[],{}
3,"March 25, 2002 3:04PM",And no manual tranny. That may not matter to y...,"[manual, tranny., may, matter, you,, certainly...","[manual, tranny., may, matter, you,, certainly...",[],{}
4,"March 25, 2002 4:44PM",One beauty of BMW 3 Series is that there are s...,"[one, beauty, bmw, 3, series, many, models, ch...","[one, beauty, bmw, 3, series, many, models, ch...",[bmw],{'bmw': 1}


In [248]:
# total frequency for each brand in all comments.
final_frequency_dict = {}

for index, row in df.iterrows():
    text_dict = row['brand_count']
    for word, frequency in text_dict.items():
        final_frequency_dict[word] = final_frequency_dict.get(word,0) + frequency

print(final_frequency_dict)

{'mercedes': 181, 'audi': 333, 'bmw': 1170, 'acura': 482, 'lexus': 353, 'buick': 19, 'toyota': 210, 'ford': 110, 'chrysler': 44, 'honda': 330, 'cadillac': 84, 'infiniti': 329, 'lincoln': 44, 'nissan': 222, 'volvo': 110, 'volkswagen': 143, 'chevrolet': 71, 'hyundai': 93, 'suzuki': 11, 'porsche': 73, 'hyundai,': 9, 'pontiac': 37, 'dodge': 43, 'mazda': 58, 'hyundai.': 12, 'subaru': 185, 'mitsubishi': 16, 'saturn': 8, 'kia': 19, 'mercury': 4, 'nissan.': 8}


In [249]:
#Top 5 brand mentions
top_5_brands=pd.Series(final_frequency_dict).sort_values(ascending=False)[:5].reset_index()
top_5_brands
top_5=pd.DataFrame(top_5_brands)
top_5.columns = ['Brand','count']
top_5

,Brand,count
0,bmw,1170
1,acura,482
2,lexus,353
3,audi,333
4,honda,330


<div class="alert alert-block alert-success">
<b>These are the top 5 car brands mentioned in the posts:</b>
    
1. BMW
2. Acura
3. Lexus
4. Audi
5. Honda
</div>

### Task 4: Identify the  brands being talked about together?


>We use 'Lift' in the context of co-mentions of car brands in a post. It is used to measure the strength of the association between two car brands based on how frequently they are mentioned together, compared to how often they are mentioned independently.

>A lift value greater than 1 would suggest a strong association between the two car brands in social media discussions, meaning that the mention of one brand has a higher chance of coinciding with the mention of the other, beyond what would be expected if their mentions were independent. 

In [290]:
#We need to create brand pairs and look for their co-mentions in the posts.
from itertools import combinations
brands = top_5['Brand'].tolist()
brand_pairs = list(combinations(brands, 2))
brand_pairs

[('bmw', 'acura'),
 ('bmw', 'lexus'),
 ('bmw', 'audi'),
 ('bmw', 'honda'),
 ('acura', 'lexus'),
 ('acura', 'audi'),
 ('acura', 'honda'),
 ('lexus', 'audi'),
 ('lexus', 'honda'),
 ('audi', 'honda')]

In [286]:
#co-mentions
d = {}
for i in brand_pairs:
    for index, row in df.iterrows():
        l = row['brand_replaced_list']
        if i[0] in l and i[1] in l:
            idx_b1=[]
            for j in range(len(l)):
                 if l[j] == i[0]:
                        idx_b1.append(j) # get all the indices with mention of brand 1 in a post
            idx_b2=[]
            for j in range(len(l)):
                 if l[j] == i[1]:
                        idx_b2.append(j) # get all the indices with mention of brand 2 in a post
                        
            break_outer_loop = False
            for x in idx_b1:
                for y in idx_b2:
                    if abs(x - y)<=15:  # Consider the brands as co mentions only when they are within 15 words from each other
                        d[i] = d.get(i, 0) + 1
                        break_outer_loop = True
                        break
                if break_outer_loop:
                        break
print(d)

{('bmw', 'acura'): 107, ('bmw', 'lexus'): 112, ('bmw', 'audi'): 99, ('bmw', 'honda'): 37, ('acura', 'lexus'): 62, ('acura', 'audi'): 51, ('acura', 'honda'): 50, ('lexus', 'audi'): 40, ('lexus', 'honda'): 15, ('audi', 'honda'): 13}


In [287]:
# Create a matrix to store lift values
import numpy as np
num_brands = len(top_5)
lift_matrix = np.zeros((num_brands, num_brands))

# Create a mapping of brand names to their indices in the matrix
brand_index_mapping = {brand: index for index, brand in enumerate(top_5['Brand'])}

# Calculate lift values and populate the matrix
for pair, count in d.items():
    brand1, brand2 = pair
    index1 = brand_index_mapping[brand1]
    index2 = brand_index_mapping[brand2]
    brand1_count = top_5.iloc[index1]['count']
    brand2_count = top_5.iloc[index2]['count']
    
    lift = 5000*count / (brand1_count * brand2_count)
    lift_matrix[index1][index2] = lift
    lift_matrix[index2][index1] = lift  # Since lift is symmetric

# Create a DataFrame from the lift matrix
lift_df = pd.DataFrame(lift_matrix, columns=top_5['Brand'], index=top_5['Brand'])

# Print the lift matrix
lift_df

Brand,bmw,acura,lexus,audi,honda
Brand,,,,,
bmw,0.000000,0.948682,1.355899,1.270501,0.479150
acura,0.948682,0.000000,1.821965,1.588726,1.571734
lexus,1.355899,1.821965,0.000000,1.701418,0.643832
audi,1.270501,1.588726,1.701418,0.000000,0.591501
honda,0.479150,1.571734,0.643832,0.591501,0.000000


In [288]:
mask = np.triu(np.ones(lift_df.shape), k=0)
brand_lifts = lift_df.where(mask == 1, "")
brand_lifts

Brand,bmw,acura,lexus,audi,honda
Brand,,,,,
bmw,0.0,0.948682,1.355899,1.270501,0.479150
acura,,0.0,1.821965,1.588726,1.571734
lexus,,,0.0,1.701418,0.643832
audi,,,,0.0,0.591501
honda,,,,,0.000000


<div class="alert alert-block alert-success">

<b>Insights:</b>

1. The lift value between Acura and Lexus is the highest in the matrix (1.821965), suggesting that these two brands are very frequently mentioned together more than would be expected by chance. This could indicate that they are seen as direct competitors or that consumers consider them together when shopping for a car.

2. BMW has highest lift with Lexus indicating that these two brands are frequenty mentioned together. This could  reflect a competitive relationship between these two brands perceived by the customers.

3. Honda's low lift values with luxury brands suggest an opportunity to emphasize its unique selling propositions, such as reliability or value for money, to differentiate from luxury car brands.
 </div>

### Task 5: Identify the top 5 most frequently mentioned attributes of cars in the discussions

In [255]:
df['without sw']
#creating list of all the words with their frequency
attribute_list=[]
def get_attribute(token_list):
    for i in token_list:
        attribute_list.append(i.lower())

df['without sw'].map(get_attribute)

#word frequency
attribute_counts=pd.Series(attribute_list).value_counts()
#exporting the word frequency to csv
pd.Series(attribute_list).value_counts().to_csv('attribute_list.csv')

<div class="alert alert-block alert-success">    
<b>Top 5 attributes:</b>
    
1. Performance	 
2. Luxury	
3. Engine	
4. Price	
5. Interior
</div>

 ### Task 6: Identify how strongly are these attributes associated with the car brands
 
>We want to focus on positive mentions of brands and attributes hence, we perform Sentiment Analysis on the comments.For this purpose, we use VADER which is a pre trained model.

>The VADER (Valence Aware Dictionary and sentiment Reasoner) sentiment score is a numerical score of sentiment strength that ranges from -1 to 1. It's generated by analyzing text to determine the sentiment expressed within it.

>Here's what the scores generally indicate:

>1. Positive Sentiment: VADER score greater than 0.05. The text is considered to have a positive sentiment.
>2. Neutral Sentiment: VADER score between -0.05 and 0.05. The text is considered to have a neutral sentiment.
>3. Negative Sentiment: VADER score less than -0.05. The text is considered to have a negative sentiment.

In [256]:
import nltk
nltk.download('vader_lexicon')

from nltk.sentiment import SentimentIntensityAnalyzer

# Initialize the VADER sentiment intensity analyzer
sia = SentimentIntensityAnalyzer()

# Function to calculate VADER compound sentiment score
def vader_sentiment_score(tokens):
    # Join the list of tokens into a single string
    text = ' '.join(tokens)
    # Calculate VADER compound score
    return sia.polarity_scores(text)['compound']

# Apply the function to the 'tokens' column
df['vader_sentiment'] = df['brand_replaced_list'].apply(vader_sentiment_score)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/anmolagrawal/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [257]:
df1=df[df['vader_sentiment']>0.05].copy() #extracting positive sentiments

top_5_brands=list(top_5['Brand'])
features=['performance','luxury','engine','price','interior']

brand_attribute_pairs = [(item1, item2) for item1 in top_5_brands for item2 in features]
brand_attribute_pairs

[('bmw', 'performance'),
 ('bmw', 'luxury'),
 ('bmw', 'engine'),
 ('bmw', 'price'),
 ('bmw', 'interior'),
 ('acura', 'performance'),
 ('acura', 'luxury'),
 ('acura', 'engine'),
 ('acura', 'price'),
 ('acura', 'interior'),
 ('lexus', 'performance'),
 ('lexus', 'luxury'),
 ('lexus', 'engine'),
 ('lexus', 'price'),
 ('lexus', 'interior'),
 ('audi', 'performance'),
 ('audi', 'luxury'),
 ('audi', 'engine'),
 ('audi', 'price'),
 ('audi', 'interior'),
 ('honda', 'performance'),
 ('honda', 'luxury'),
 ('honda', 'engine'),
 ('honda', 'price'),
 ('honda', 'interior')]

In [280]:
#co-mention counts
attribute_brands = {}
for i in brand_attribute_pairs:
    for index, row in df1.iterrows():
        l = row['brand_replaced_list']
        if i[0] in l and i[1] in l:
             idx_b1=[]
             for j in range(len(l)):
                 if l[j] == i[0]:
                        idx_b1.append(j)
             idx_b2=[]
             for j in range(len(l)):
                 if l[j] == i[1]:
                        idx_b2.append(j)
                        
             break_outer_loop = False
             for x in idx_b1:
                for y in idx_b2:
                    if abs(x - y)<=60:
                        attribute_brands [i] = attribute_brands.get(i, 0) + 1
                        break_outer_loop = True
                        break
                if break_outer_loop:
                        break
           # attribute_brands [i] = attribute_brands.get(i, 0) + 1
         

print(attribute_brands )

{('bmw', 'performance'): 148, ('bmw', 'luxury'): 87, ('bmw', 'engine'): 89, ('bmw', 'price'): 109, ('bmw', 'interior'): 69, ('acura', 'performance'): 61, ('acura', 'luxury'): 52, ('acura', 'engine'): 36, ('acura', 'price'): 45, ('acura', 'interior'): 33, ('lexus', 'performance'): 43, ('lexus', 'luxury'): 49, ('lexus', 'engine'): 22, ('lexus', 'price'): 31, ('lexus', 'interior'): 36, ('audi', 'performance'): 34, ('audi', 'luxury'): 31, ('audi', 'engine'): 26, ('audi', 'price'): 28, ('audi', 'interior'): 36, ('honda', 'performance'): 47, ('honda', 'luxury'): 37, ('honda', 'engine'): 17, ('honda', 'price'): 34, ('honda', 'interior'): 22}


In [273]:
attributes=['performance','luxury','engine','price','interior']

def attr_count(l):
    d={}
    for i in l:
        if i in attributes:
            d[i]=1
    return d
df['attribute_count']=df['brand_replaced_list'].map(attr_count)
df

,date,message,without sw,brand_replaced_list,brand_list,brand_count,vader_sentiment,attribute_count
0,"March 24, 2002 9:54PM",I personally think that with a few tweaks the ...,"[personally, think, tweaks, c320, could, also,...","[personally, think, tweaks, c320, could, also,...","[mercedes, audi, bmw, acura, lexus, buick, toy...","{'mercedes': 1, 'audi': 1, 'bmw': 1, 'acura': ...",0.9983,"{'luxury': 1, 'interior': 1, 'engine': 1}"
1,"March 24, 2002 11:06PM",I am debating a new purchase and these two are...,"[debating, new, purchase, two, top, list., pri...","[debating, new, purchase, two, top, list., pri...","[honda, lexus]","{'honda': 1, 'lexus': 1}",0.9419,{'price': 1}
2,"March 25, 2002 9:02AM","Great handling, RWD, excellent engine and the ...","[great, handling,, rwd,, excellent, engine, be...","[great, handling,, rwd,, excellent, engine, be...",[],{},0.9623,{'engine': 1}
3,"March 25, 2002 3:04PM",And no manual tranny. That may not matter to y...,"[manual, tranny., may, matter, you,, certainly...","[manual, tranny., may, matter, you,, certainly...",[],{},0.7184,{}
4,"March 25, 2002 4:44PM",One beauty of BMW 3 Series is that there are s...,"[one, beauty, bmw, 3, series, many, models, ch...","[one, beauty, bmw, 3, series, many, models, ch...",[bmw],{'bmw': 1},0.5228,"{'engine': 1, 'performance': 1}"
...,...,...,...,...,...,...,...,...
4995,"January 23, 2006 8:51PM",I agree with blueguydotcom. The IS350 is one f...,"[agree, blueguydotcom., is350, one, fast, ride...","[agree, blueguydotcom., is350, one, fast, ride...",[honda],{'honda': 1},0.8854,{'performance': 1}
4996,"January 23, 2006 10:29PM",Don't have any interest in aftermarket tuners....,"[interest, aftermarket, tuners., 3.5, 3.0tt, b...","[interest, aftermarket, tuners., 3.5, 3.0tt, b...",[bmw],{'bmw': 1},0.4215,{}
4997,"January 24, 2006 6:39PM",>Don't have any interest in aftermarket tuners...,"[>don't, interest, aftermarket, tuners, consid...","[>don't, interest, aftermarket, tuners, consid...",[bmw],{'bmw': 1},-0.5803,{}
4998,"January 24, 2006 6:42PM",AMG i have no use for...it's MB afterall. As f...,"[amg, use, for...it's, mb, afterall., m,, actu...","[amg, use, for...it's, mb, afterall., m,, actu...",[bmw],{'bmw': 1},-0.2047,{}


In [274]:
final_attribute_frequency = {}

for index, row in df.iterrows():
    text_dict = row['attribute_count']
    for word, frequency in text_dict.items():
        final_attribute_frequency[word] = final_attribute_frequency.get(word,0) + frequency

print(final_attribute_frequency)

{'luxury': 371, 'interior': 281, 'engine': 330, 'price': 326, 'performance': 517}


In [226]:
attribute_brand_counts=top_5.copy()
attribute_brand_counts

,Brand,count
0,bmw,1170
1,acura,482
2,lexus,353
3,audi,333
4,honda,330


In [1]:
import pandas as pd

# Sample dataframes
data1 = {'Attribute': ['performance', 'luxury', 'engine', 'price', 'interior'],
         'Count': [517, 371, 330, 326, 281]}

data2 = {'Brand': ['bmw', 'acura', 'lexus', 'audi', 'honda'],
         'Count': [1170,482,353,333,330]}

df_attribute = pd.DataFrame(data1)
df_brand = pd.DataFrame(data2)

# Create a dictionary for attribute_brands

attribute_brands={('bmw', 'performance'): 148, ('bmw', 'luxury'): 87, ('bmw', 'engine'): 89, ('bmw', 'price'): 109, ('bmw', 'interior'): 69, ('acura', 'performance'): 61, ('acura', 'luxury'): 52, ('acura', 'engine'): 36, ('acura', 'price'): 45, ('acura', 'interior'): 33, ('lexus', 'performance'): 43, ('lexus', 'luxury'): 49, ('lexus', 'engine'): 22, ('lexus', 'price'): 31, ('lexus', 'interior'): 36, ('audi', 'performance'): 34, ('audi', 'luxury'): 31, ('audi', 'engine'): 26, ('audi', 'price'): 28, ('audi', 'interior'): 36, ('honda', 'performance'): 47, ('honda', 'luxury'): 37, ('honda', 'engine'): 17, ('honda', 'price'): 34, ('honda', 'interior'): 22}


# Create a dataframe from the dictionary
df_lift = pd.DataFrame(attribute_brands.items(), columns=['Brand_Attribute', 'Count'])

# Split the 'Brand_Attribute' column into 'Brand' and 'Attribute'
df_lift[['Brand', 'Attribute']] = pd.DataFrame(df_lift['Brand_Attribute'].tolist(), index=df_lift.index)

# Merge df_lift with df_attribute to get attribute counts
df_lift = df_lift.merge(df_attribute, on='Attribute', how='left')

# Merge df_lift with df_brand to get brand counts
df_lift = df_lift.merge(df_brand, on='Brand', how='left')

# Calculate lift using the formula: lift = (Count(Brand, Attribute) * N) / (Count(Brand) * Count(Attribute))


df_lift['Lift'] = (df_lift['Count_x'] * 5000) / (df_lift['Count_y'] * df_lift['Count'])

# Display the dataframe with lift values
df_lift[['Brand', 'Attribute', 'Lift']]


pivot_table = pd.pivot_table(df_lift, values='Lift', index='Brand', columns='Attribute')

# Display the pivot table
pivot_table.T

Brand,acura,audi,bmw,honda,lexus
Attribute,,,,,
engine,1.131648,1.183001,1.152551,0.780533,0.944287
interior,1.218234,1.923632,1.049366,1.186240,1.814644
luxury,1.453960,1.254624,1.002143,1.511068,1.870757
performance,1.223946,0.987448,1.223363,1.377410,1.178076
price,1.431917,1.289633,1.428871,1.580219,1.346913


<div class="alert alert-block alert-success">
<b>Insights</b>:

1. Audi and BMW have the highest lift values for engine-related positive comments, indicating that customers particularly appreciate their engine quality or performance. Honda has a notably lower lift value, suggesting less association with engine quality in positive sentiments.

2. Lexus and Audi leads significantly in positive associations with interior. This indicates that when people make positive comments, they often highlight the interior quality of these brands. Acura and BMW have lower lift values, suggesting this may not be a standout attribute in positive mentions.

3. Lexus has the highest lift value for luxury, reinforcing its image as a luxury brand in the minds of consumers. 

4. Honda stands out with the highest lift value for performance, suggesting that positive sentiments about Honda frequently reference its performance. This is interesting as it contrasts with the lower lift value for the engine, indicating that Honda's overall performance is appreciated beyond just the engine.

5. Honda has the highest lift value for price, which may indicate that customers feel the pricing of Honda cars is well-justified or they see great value in the brand.


<b>Business Implications</b>:

1. Marketing Strategy: Marketing efforts should align with these strengths. Lexus can focus on their luxury status, while Honda can highlight their performancece in advertising campaigns.

2. Brand Positioning: Lexus should continue to position itself in the luxury segment, leveraging its strong association with luxury in customer sentiments. Honda's emphasis could be on the performance aspect, differentiating itself from competitors.

3. Pricing Communication: Honda's high lift value for price indicates that their pricing is seen as favorable. Honda can capitalize on this by promoting its cars as offering excellent 'performance' for money. Audi and BMW should ensure their pricing strategies are transparent and communicate the value customers receive.

4. Customer Experience: For brands with lower lift values in certain attributes, such as Acura and BMW's interior, there may be opportunities to improve customer experience in these areas or to communicate existing quality more effectively.
</div>